# Reproducing in-text results 6 and 7

These results are variants on the scenario from Figure 2B, when the IAT is 3 minutes (170 arrivals/day) and the NCD nurse utilisation exceeds 100% (is at 123%). The variants are to:

* Assign administrative work to the staff nurse
* Have the staff nurse complete 10% of NCD checks

## Set up

In [1]:
# To run model
import PHC

# To import results and produce figures
from reproduction_helpers import process_results
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# To speed up run time
from multiprocessing import Pool

# Additional package to record runtime of this notebook
import time
start = time.time()

In [2]:
# Paths to save results to
output_folder = 'outputs'
txt6_path = os.path.join(output_folder, 'intext6.csv')
txt7_path = os.path.join(output_folder, 'intext7.csv')

## Run model

In [3]:
dict_list = [
    {
        'OPD_iat': 3,
        'rep_file': 'in67_base.xls',
        'output_full_results': True,
        'full_file': 'full_in67_base.xlsx'
    },
    {
        'OPD_iat': 3,
        'admin_ncd_to_staff': True,
        'rep_file': 'in6_base_admin.xls',
        'output_full_results': True,
        'full_file': 'full_in6_base_admin.xlsx'
    },
    {
        'OPD_iat': 3,
        'opd_10p_ncd_to_staff': True,
        'rep_file': 'in7_base_appointment.xls',
        'output_full_results': True,
        'full_file': 'full_in7_base_appointment.xlsx'
    },
    {
        'OPD_iat': 3,
        'admin_ncd_to_staff': True,
        'opd_10p_ncd_to_staff': True,
        'rep_file': 'in7_base_admin_appointment.xls',
        'output_full_results': True,
        'full_file': 'full_in7_base_admin_appointment.xlsx'
    }
]

In [4]:
# Append 's_' to all items
for i, d in enumerate(dict_list):
    dict_list[i] = {f's_{k}': v for k, v in d.items()}

In [5]:
# Wrapper function to allow input of dictionary with pool
def wrapper(d):
    return PHC.main(**d)

# Create a process pool that uses all CPUs
with Pool() as pool:
    # Run PHC.main() using each of inputs from config
    pool.map(wrapper, dict_list)

 No of replications done 0
 No of replications done 0
 No of replications done 0
 No of replications done 0
 No of replications done 1
 No of replications done 1
 No of replications done 1
 No of replications done 1
 No of replications done 2
 No of replications done 2
 No of replications done 2
 No of replications done 2
 No of replications done 3
 No of replications done 3
 No of replications done 3
 No of replications done 3
 No of replications done 4
 No of replications done 4
 No of replications done 4
 No of replications done 4
 No of replications done 5
 No of replications done 5
 No of replications done 5
 No of replications done 5
 No of replications done 6
 No of replications done 6
 No of replications done 6
 No of replications done 6
 No of replications done 7
 No of replications done 7
 No of replications done 7
 No of replications done 7
 No of replications done 8
 No of replications done 8
 No of replications done 8
 No of replications done 8
 No of replications done 9
 

## Process results

In [6]:
# Process results
data = process_results([i['s_rep_file'] for i in dict_list], xls=True)

In [7]:
def get_util(df):
    '''
    Gets NCD utilisation and transforms the dataframe to aid readability

    Parameters:
    -----------
    df : pandas DataFrame
        Dataframe with mean results for different outputs

    Returns:
    --------
    util : pandas DataFrame
        Dataframe with the NCD utilisation in each scenario
    '''
    # Get NCD utilisation
    util = round(pd.DataFrame(df.loc['NCD occ']).T, 3)

    # Rename index and row
    util = util.rename_axis('Output')
    util = util.rename({'NCD occ': 'NCD nurse utilisation'})

    # Rename columns
    util = util.rename({
        'in67_base': 'Normal',
        'in6_base_admin': 'Admin from NCD to staff nurse',
        'in7_base_appointment': '10% OPD appointments from NCD to staff nurse',
        'in7_base_admin_appointment': (
            'Admin and 10% OPD appointments from NCD to staff nurse')}, axis=1)

    return util

## Get in-text result 6

In [8]:
txt6 = get_util(data[['in67_base', 'in6_base_admin']])

# Save and display results
txt6.to_csv(txt6_path)
txt6

,Normal,Admin from NCD to staff nurse
Output,,
NCD nurse utilisation,1.233,1.0


## Get in-text result 7

In [9]:
txt7 = get_util(data)

# Save and display results
txt7.to_csv(txt7_path)
txt7

,Normal,Admin from NCD to staff nurse,10% OPD appointments from NCD to staff nurse,Admin and 10% OPD appointments from NCD to staff nurse
Output,,,,
NCD nurse utilisation,1.233,1.0,1.071,0.825


## Get full results files

In [25]:
full_files = [i['s_full_file'] for i in dict_list]
for f in full_files:
    full_result = pd.read_excel(
    os.path.join(output_folder, f), header=None).rename(columns={
        0: 'full_outcome',
        1: 'full_result'})
    display(full_result[full_result['full_outcome'] == 'NCD Nurse Occupancy'])

,full_outcome,full_result
30,NCD Nurse Occupancy,1.23


,full_outcome,full_result
30,NCD Nurse Occupancy,1.0


,full_outcome,full_result
30,NCD Nurse Occupancy,1.07


,full_outcome,full_result
30,NCD Nurse Occupancy,0.82


## Run time

In [10]:
# Find run time in seconds
end = time.time()
runtime = round(end-start)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime//60}m {runtime%60}s')

Notebook run time: 2m 52s
